In [ ]:
#importing the necessary libraries and dependencies
import pandas as pd
import numpy as np
import seaborn as sns;
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from numpy import array
from keras.models import Sequential
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import time

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('val%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('val%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('val%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# loading the data into the dataframe
dataset = pd.read_csv('/content/drive/MyDrive/AQI/Data.csv', header=0, index_col=0)
dataset.head()

In [ ]:
# dataset.rename(columns = {'Unnamed: 0':'ID'}, inplace = True)

In [ ]:
# #number of rows and columns in the dataset
print(dataset.shape)

In [ ]:
dataset=dataset.dropna()

In [ ]:
# #statistical information about columns
print(dataset.describe())

In [ ]:
# df=dataset.describe()
# df.to_excel('/content/drive/MyDrive/AQI/Description.xlsx')

In [ ]:
dataset.drop({'wd',}, axis = 1, inplace = True)
dataset.drop({'station',}, axis = 1, inplace = True)
dataset.drop({'date',}, axis = 1, inplace = True)
dataset.drop({'AQI',}, axis = 1, inplace = True)
# dataset.drop({'dateInt',}, axis = 1, inplace = True)
# dataset.drop({'AQI_numeric',}, axis = 1, inplace = True)

In [ ]:
dataset.head()

In [ ]:
# dataset = pd.DataFrame(dataset, columns=["PM2.5", "PM10", "SO2", "NO2", "CO", "O3", "PRES", "DEWP", "RAIN", "WSPM",])
# # dataset.plot.box()
# dataset.plot( kind="area", figsize=(15, 12))
# plt.show()

In [ ]:
# #checking how many null values are in each column
# dataset.drop({'wd',}, axis = 1, inplace = True)
# dataset.drop({'station',}, axis = 1, inplace = True)
# dataset.drop({'date',}, axis = 1, inplace = True)
# dataset.drop({'AQI',}, axis = 1, inplace = True)
# dataset.drop({'TEMP',}, axis = 1, inplace = True)
# dataset.drop({'PRES',}, axis = 1, inplace = True)
# dataset.drop({'DEWP',}, axis = 1, inplace = True)
# dataset.drop({'RAIN',}, axis = 1, inplace = True)
# dataset.drop({'WSPM',}, axis = 1, inplace = True)
# dataset.drop({'dateInt',}, axis = 1, inplace = True)
# dataset.drop({'AQI_numeric',}, axis = 1, inplace = True)

In [ ]:
# #number of rows and columns in the dataset
print(dataset.shape)

Just doing `df.dropna()` drops all the NaN values only for the current execution of the cell. If you do the above `df.isnull().sum()` now, you can see that null values still persists. You can solve this by assigning the obtained output of
`df.dopna()` to the variable `df` which stores our data (dataframe)

In [ ]:
# correlations = dataset.corr()
# fig, ax = plt.subplots(figsize=(15,15))
# sns.heatmap(correlations, vmax=1.0, center=0, fmt='.2f', square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .70})
# plt.show();

In [ ]:
# g = sns.pairplot(dataset)

In [ ]:
# dataset.plot(kind='density', subplots=True, layout=(4,4), sharex=False, figsize=(30,20))
# plt.show()

In [ ]:
# values = dataset.values
# # integer encode direction
# encoder = LabelEncoder()
# values[:,4] = encoder.fit_transform(values[:,4])
# print(values)
# pd.DataFrame(values).to_excel('/content/drive/MyDrive/AQI/IntergerEncoding.xlsx')

In [ ]:

# # ensure all data is float
# values = values.astype('float32')
# np.savetxt('/content/drive/MyDrive/AQI/my_data.csv', values, delimiter=',')
# # values.to_csv(r'/content/drive/MyDrive/AQI/my_data.csv', index=False)
# print(values)

# pd.DataFrame(values).to_excel('/content/drive/MyDrive/AQI/FloatConversion.xlsx')

In [ ]:
# reframed = series_to_supervised(values, 1, 1)
# # split into train and test sets
# values = reframed.values
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled_features = scaler.fit_transform(values[:,:-1])
# scaled_label = scaler.fit_transform(values[:,-1].reshape(-1,1))
# values = np.column_stack((scaled_features, scaled_label))
# print(values)
# pd.DataFrame(values).to_excel('/content/drive/MyDrive/AQI/MinMaxScaling.xlsx')

In [ ]:
values = dataset.values
# integer encode direction
encoder = LabelEncoder()
values[:,4] = encoder.fit_transform(values[:,4])
# ensure all data is float
values = values.astype('float32')
reframed = series_to_supervised(values, 1, 1)
reframed.drop(reframed.columns[[-7,-6,-5,-4,-3,-2,-1]], axis=1, inplace=True)
print(reframed.head())

# split into train and test sets
values = reframed.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(values[:,:-1])
scaled_label = scaler.fit_transform(values[:,-1].reshape(-1,1))
values = np.column_stack((scaled_features, scaled_label))


In [ ]:
n_train_hours = 365 * 24 * 40
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
# split into input and outputs
# features take all values except the var1
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)


In [ ]:
from google.colab import files

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(17,8))
plt.plot(train_y, label='train_y', color='#0096FF',)
plt.plot(test_y, label='test_y',color='#1F51FF',)

plt.legend(['train', 'test'], loc='upper left')

plt.savefig("traintest.pdf")
files.download("traintest.pdf")
#plt.show()

In [ ]:
# design network
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(1, 18)),
    tf.keras.layers.Dense(units=1)
])
#keras.optimizers.SGD(lr=0.001, decay=1e-5, momentum=1.0, nesterov=False)
#keras.optimizers.SGD(lr=0.01, decay=1e-5, momentum=0.9, nesterov=True) #good

#keras.optimizers.RMSprop(learning_rate=0.01, rho=0.9)

keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.9, amsgrad=False)
model.compile(loss='mae', optimizer='adam', metrics=['accuracy'] )
tf.keras.utils.plot_model(model=model, show_shapes=True)
tf.keras.utils.plot_model(model=model, show_shapes=True)

In [ ]:
model.summary()

In [ ]:
start = time.time()

# Fit the network
history = model.fit(train_X, train_y, epochs=10, validation_data=(test_X, test_y), verbose=1, shuffle=False)

# Plot history
plt.figure(figsize=(17, 8))
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

# Make predictions
yhat = model.predict(test_X)

# Reshape test_X if needed
if len(test_X.shape) == 3:
    test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))

# Invert scaling for forecast
inv_yhat = np.concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:, 0]

# Invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = np.concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:, 0]

end = time.time()
print('This took {} seconds.'.format(end - start))

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
mae = tf.keras.metrics.mean_absolute_error(inv_y, inv_yhat)
inv_y = scaler.inverse_transform(inv_y.reshape(-1, 1))
inv_yhat = scaler.inverse_transform(inv_yhat.reshape(-1, 1))


In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
# calculate matrics
mse = mean_squared_error(test_y, yhat)
mae = mean_absolute_error(test_y, yhat)
mape = mean_absolute_percentage_error(test_y, yhat)


print('MSE: %.5f' % mse)
print('MAE : %.5f' % mae)
print('MAPE : %.5f' % mape)

In [ ]:
mse = mean_squared_error(test_y, yhat)
print('MSE: %.5f' % mse)


In [ ]:
from google.colab import files

In [ ]:
def plot_predicted(predicted_data, true_data):
    fig, ax = plt.subplots(figsize=(17,8))
    ax.set_title('Prediction vs. Actual after 10 epochs of training')
    ax.plot(true_data, label='True Data', color='green', linewidth='3')

    ax.plot(predicted_data, label='Prediction', color='orange', linewidth='2')
    plt.legend()
    plt.savefig("LSTMPre.pdf")
    files.download("LSTMPre.pdf")
    plt.show()
plot_predicted(inv_yhat[:1000,], inv_y[:1000,])
print('Root Mean Squared Error: {:.4f}'.format(rmse))
print("R2 score : %.2f" % r2_score(inv_y,inv_yhat))


In [ ]:
# summarize history for loss
pyplot.figure(figsize=(17,8))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.savefig("loss.pdf")
files.download("loss.pdf")
plt.show()

In [ ]:
t=history.history['val_loss']
np.average(t)

In [ ]:

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig("Accuracy.pdf")
files.download("Accuracy.pdf")
plt.show()